In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\vaibh\\OneDrive\\Desktop\\full_model\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vaibh\\OneDrive\\Desktop\\full_model'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    """
    Data Ingestion Configuration
    """
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path
    

In [10]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml , create_directories


In [11]:
class ConfigurationManager:

    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Method to get the data ingestion configuration
        """
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
        

In [15]:
#component
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [36]:
class DataIngestion:

    def __init__(self , config :DataIngestionConfig):
        self.config = config 
    
    def dowload_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers  = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers} ")

        else:
            logger.info(f"File already exists at {self.config.local_data_file} of size {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")




In [38]:
## pipeline 

try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.dowload_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-10 15:16:47,541: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-10 15:16:47,544: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-10 15:16:47,547: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-10 15:16:47,549: INFO: common: created directory at: artifacts]
[2025-06-10 15:16:47,551: INFO: common: created directory at: artifacts/data_ingestion]


[2025-06-10 15:16:48,574: INFO: 294664432: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 268038
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "9c002cf34d808ddea007489b47eb0a21b85050243bd6b36b85cb4d8f208b7fff"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 205B:3CBFD3:175988:31AF56:6847FF07
Accept-Ranges: bytes
Date: Tue, 10 Jun 2025 09:46:48 GMT
Via: 1.1 varnish
X-Served-By: cache-del21744-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1749548808.841170,VS0,VE338
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: e10e0387d31c12b98c11549b5d09d7512f0e233e
Expires: Tue, 10 Jun 2025 09:51:48 GMT
Source-Age: 0

 ]
[2025-06-10 15:16:48,617: INFO: 294664432: Extract